<a href="https://colab.research.google.com/github/omkarwazulkar/GoogleColab/blob/main/IMDB_DataClassification_BERT_HF_Train_Push_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERT Training**

In [19]:
pip install transformers datasets

In [20]:
from datasets import load_dataset

dataset = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [22]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [23]:
train_dataset = dataset['train'].shuffle(seed=42).select(range(2000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(2000))

In [24]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [25]:
train_dataset[0]['text']

'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'

In [26]:
train_dataset[0]['label']

1

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [28]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__bool__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_call_one',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_pad',
 '_pad_token_type_id',
 '_processor_class',
 '_save_pretrained',
 '

In [29]:
print("Vocab Size:", tokenizer.vocab_size)
print("Pad Token:", tokenizer.pad_token)
print("CLS Token:", tokenizer.cls_token)
print("SEP Token:", tokenizer.sep_token)
print("Model Max Length:", tokenizer.model_max_length)

Vocab Size: 30522
Pad Token: [PAD]
CLS Token: [CLS]
SEP Token: [SEP]
Model Max Length: 512


In [ ]:
token_lengths = [len(tokenizer(text, truncation=False)['input_ids']) for text in train_dataset['text']]
print("Max:", max(token_lengths), "Average:", sum(token_lengths)/len(token_lengths))

Token indices sequence length is longer than the specified maximum sequence length for this model (936 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def tokenize(batch):
    return tokenizer(
        batch['text'],
        padding='max_length',
        truncation=True,
        max_length=256)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
type(train_dataset)

In [ ]:
train_dataset

In [ ]:
train_dataset[0]['attention_mask'].count(0)

In [ ]:
train_dataset[0]['input_ids'].count(0)

In [ ]:
set(train_dataset['label'])

In [ ]:
train_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

test_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

In [ ]:
train_dataset[0]

In [ ]:
len(test_dataset)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # Positive/Negative
).to(device)

In [ ]:
device

In [ ]:
model

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = CrossEntropyLoss()

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import torch
import numpy as np
from sklearn.metrics import accuracy_score

k = 5  # number of folds
batch_size = 16

kfold = KFold(n_splits=k, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n----- FOLD {fold+1} / {k} -----")

    # Create subsets for this fold
    train_subset = Subset(train_dataset, train_idx)
    val_subset   = Subset(train_dataset, val_idx)

    # Create dataloaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_subset, batch_size=batch_size)

    # Recreate model, optimizer, loss function for each fold
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    criterion = torch.nn.CrossEntropyLoss()

    epochs = 3  # choose what you want

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Fold {fold+1}, Epoch {epoch+1}, Train Loss: {avg_loss:.4f}")

    # Validation for this fold
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    fold_acc = accuracy_score(all_labels, all_preds)
    fold_results.append(fold_acc)

    print(f"Fold {fold+1} Accuracy: {fold_acc:.4f}")

# Final averaged accuracy
print("\n===== FINAL CROSS-VALIDATION RESULT =====")
print("Accuracies:", fold_results)
print("Mean Accuracy:", np.mean(fold_results))

In [ ]:
from sklearn.metrics import accuracy_score

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch+1}, Train Loss: {avg_loss:.4f}, Test Accuracy: {acc:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(all_labels, all_preds)
print(acc)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(all_labels, all_preds))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# F1-score (macro and weighted)
f1_macro = f1_score(all_labels, all_preds, average='macro')
f1_weighted = f1_score(all_labels, all_preds, average='weighted')

print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")

all_probs = [float(p) for p in all_preds]

In [ ]:
# ROC-AUC Score
roc_auc = roc_auc_score(all_labels, all_probs)
print(f"ROC-AUC: {roc_auc:.4f}")

# ROC Curve
fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
roc_auc_value = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_value:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [9]:
texts = [
    "I loved this movie!",
    "This was awful.",
    "Amazing storyline and acting.",
    "I hate everything about this.",
    "What a beautiful film!",
    "Terrible direction.",
    "Absolutely fantastic experience!",
    "Super boring.",
    "One of the best movies ever.",
    "I will never watch this again.",
    "Loved every moment of it.",
    "Waste of time.",
    "Great visuals and music.",
    "The plot was predictable.",
    "Outstanding performance!"
]

In [ ]:
model.eval()
inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    for text, pred in zip(texts, preds):
        print(f"Text: {text} --> Sentiment: {'Positive' if pred==1 else 'Negative'}")

In [ ]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")

# **Push Custom-Trained Model To HF**

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Example: changing labels
model.config.id2label = {0: "Negative", 1: "Positive"}
model.config.label2id = {"Negative": 0, "Positive": 1}

In [ ]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository

model.push_to_hub("my-model-name")
tokenizer.push_to_hub("my-model-name")

# **Loading Custom-Trained HF Model**

In [ ]:
!pip install huggingface_hub transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="omkarwazulkar/my-model-name")

In [ ]:
results = pipe(texts)

for text, res in zip(texts, results):
    print(f"Text: {text}")
    print(f"Prediction: {res}\n")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("omkarwazulkar/my-model-name")
_model = AutoModelForSequenceClassification.from_pretrained("omkarwazulkar/my-model-name")

In [ ]:
_model

In [ ]:
import torch

# Tokenize
inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = _model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    for text, pred in zip(texts, preds):
      print(f"Text: {text} --> Sentiment: {'Positive' if pred==1 else 'Negative'}")

# **ONNX**

In [ ]:
!pip install transformers onnx onnxruntime onnxruntime-tools

In [ ]:
!pip install optimum[onnxruntime]

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer
from pathlib import Path

model_name = "omkarwazulkar/my-model-name"
save_path = Path("onnx_model")

# 1. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Export ONNX model (new API!)
onnx_model = ORTModelForSequenceClassification.from_pretrained(
    model_name,
    export=True  # <-- Export ONNX
)

# 3. Save model + tokenizer
onnx_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np

onnx_folder = "onnx_model"   # the folder you saved earlier

tokenizer = AutoTokenizer.from_pretrained(onnx_folder)

session = ort.InferenceSession(f"{onnx_folder}/model.onnx")

In [ ]:
def predict(text):
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
        truncation=True,
        max_length=256
    )

    # ONNX model inference
    outputs = session.run(
        None,
        {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"]
        }
    )

    logits = outputs[0]
    pred_id = int(np.argmax(logits, axis=1)[0])
    return pred_id, logits

In [ ]:
id2label = {0: "Negative", 1: "Positive"}

In [ ]:
text = "I really hate this movie!"
pred_id, _ = predict(text)
print("Prediction:", id2label[pred_id])

In [ ]:
import numpy as np

def predict_batch(texts):
    inputs = tokenizer(
        texts,
        return_tensors="np",
        padding=True,
        truncation=True,
        max_length=256
    )

    outputs = session.run(
        None,
        {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"]
        }
    )

    logits = outputs[0]     # ONNX output #1
    preds = np.argmax(logits, axis=1)
    return preds, logits

In [ ]:
id2label = {0: "Negative", 1: "Positive"}

In [ ]:
pred_ids, logits = predict_batch(texts)

for text, pred in zip(texts, pred_ids):
    print(f"Text: {text}\nPrediction: {id2label[pred]}\n")

# **Post Training Quantization Of Custom-Trained HF Model**

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
import tempfile
import os

# Load Model From HF
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("omkarwazulkar/my-model-name")
hf_model = AutoModelForSequenceClassification.from_pretrained("omkarwazulkar/my-model-name").to(device)

In [4]:
for i, (name, param) in enumerate(hf_model.named_parameters()):
    print(name, param.dtype)
    if i >= 4:
      break

distilbert.embeddings.word_embeddings.weight torch.float32
distilbert.embeddings.position_embeddings.weight torch.float32
distilbert.embeddings.LayerNorm.weight torch.float32
distilbert.embeddings.LayerNorm.bias torch.float32
distilbert.transformer.layer.0.attention.q_lin.weight torch.float32


In [5]:
with tempfile.TemporaryDirectory() as tmpdir:
    fp32_path = os.path.join(tmpdir, "fp32_model.pth")
    torch.save(hf_model.state_dict(), fp32_path)
    fp32_size = os.path.getsize(fp32_path) / 1e6
    print(f"FP32 Model Size: {fp32_size:.2f} MB")

FP32 Model size: 267.86 MB


In [6]:
hf_model.cpu()
quantized_model = torch.quantization.quantize_dynamic(
    hf_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

/tmp/ipython-input-3161282142.py:2: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model = torch.quantization.quantize_dynamic(


In [7]:
with tempfile.TemporaryDirectory() as tmpdir:
    int8_path = os.path.join(tmpdir, "int8_model.pth")
    torch.save(quantized_model.state_dict(), int8_path)
    int8_size = os.path.getsize(int8_path) / 1e6
    print(f"INT8 Quantized Model Size: {int8_size:.2f} MB")

# 6️⃣ Optional: Compare
print(f"\nSize Reduction: {fp32_size / int8_size:.2f}x Smaller With INT8 Quantization")

INT8 Quantized Model Size: 138.72 MB

Size Reduction: 1.93x Smaller With INT8 Quantization


In [10]:
import torch

# Tokenize
inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = quantized_model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    for text, pred in zip(texts, preds):
      print(f"Text: {text} --> Sentiment: {'Positive' if pred==1 else 'Negative'}")

Text: I loved this movie! --> Sentiment: Positive
Text: This was awful. --> Sentiment: Negative
Text: Amazing storyline and acting. --> Sentiment: Positive
Text: I hate everything about this. --> Sentiment: Negative
Text: What a beautiful film! --> Sentiment: Positive
Text: Terrible direction. --> Sentiment: Negative
Text: Absolutely fantastic experience! --> Sentiment: Positive
Text: Super boring. --> Sentiment: Negative
Text: One of the best movies ever. --> Sentiment: Positive
Text: I will never watch this again. --> Sentiment: Negative
Text: Loved every moment of it. --> Sentiment: Positive
Text: Waste of time. --> Sentiment: Negative
Text: Great visuals and music. --> Sentiment: Positive
Text: The plot was predictable. --> Sentiment: Negative
Text: Outstanding performance! --> Sentiment: Positive
